In [1]:
import os
import pandas as pd
import SimpleITK as sitk
import glob

In [2]:
output_directory = r'C:\Users\LydiaN\OneDrive - Personal\OneDrive\Work\DigitalAssets\BICCNMorphologyInventory\u19zeng'
inventory_path = os.path.join(output_directory,'sample_inventory_with_image_info.csv')
inventory = pd.read_csv(inventory_path)
inventory.set_index('Sample ID', inplace=True)

In [3]:
example_directory = os.path.join( output_directory, 'hust_imaging_example_thumbnails')
os.makedirs( example_directory, exist_ok=True )
cols = ['hust_imaging_id','labtrack_mouse_id','full_resolution_image_path','downsampled_image_path','x_size','y_size']
examples = pd.DataFrame( columns = cols)

In [4]:
for iindex, irow in inventory.iterrows() :
    
    #if irow['HUST Sample ID'] < 201595 :
    if irow['HUST Sample ID'] < 201604 :
        continue

    # specimen input directory
    spath = os.path.join(output_directory,iindex)
    tpath = os.path.join( spath, 'green', 'raw', 'fullres')

    for fpath in glob.glob(os.path.join(tpath,'*.tif')) :
        
        print (iindex,fpath)
        bname = os.path.basename( fpath )
        
        image = sitk.ReadImage( fpath )
        
        rec = {}
        rec['hust_imaging_id'] = irow['HUST Sample ID']
        rec['labtrack_mouse_id'] = irow['Subject ID']
        rec['full_resolution_image_path'] = bname
        rec['x_size'] = image.GetSize()[0]
        rec['y_size'] = image.GetSize()[1]
        
        dname = "%d_downsampled_example.png" % rec['hust_imaging_id']
        rec['downsampled_image_path'] = dname
        opath = os.path.join( example_directory, dname )
        
        dimage = sitk.Shrink( image, (25,25))
        #dimage = sitk.IntensityWindowing( dimage, 0, 800, 0, 255 )
        #dimage = sitk.IntensityWindowing( dimage, 0, 300, 0, 255 )
        dimage = sitk.IntensityWindowing( dimage, 0, 150, 0, 255 )

        dimage = sitk.Cast( dimage, sitk.sitkUInt8 )
        sitk.WriteImage( dimage, opath, True )
        
        examples = examples.append(rec, ignore_index=True)
        break
        

mouseID_511266-201604 C:\Users\LydiaN\OneDrive - Personal\OneDrive\Work\DigitalAssets\BICCNMorphologyInventory\u19zeng\mouseID_511266-201604\green\raw\fullres\201604_05001_CH1.tif
mouseID_512894-201605 C:\Users\LydiaN\OneDrive - Personal\OneDrive\Work\DigitalAssets\BICCNMorphologyInventory\u19zeng\mouseID_512894-201605\green\raw\fullres\201605_05001_CH1.tif
mouseID_512897-201606 C:\Users\LydiaN\OneDrive - Personal\OneDrive\Work\DigitalAssets\BICCNMorphologyInventory\u19zeng\mouseID_512897-201606\green\raw\fullres\201606_05001_CH1.tif


In [5]:
manifest_directory = os.path.join( example_directory, 'manifests')
os.makedirs( manifest_directory, exist_ok = True )
examples.to_csv(os.path.join( manifest_directory, 'examples_dummy.csv'), index=False )

In [6]:
examples

,hust_imaging_id,labtrack_mouse_id,full_resolution_image_path,downsampled_image_path,x_size,y_size
0,201604,511266,201604_05001_CH1.tif,201604_downsampled_example.png,28523,22018
1,201605,512894,201605_05001_CH1.tif,201605_downsampled_example.png,28627,25005
2,201606,512897,201606_05001_CH1.tif,201606_downsampled_example.png,30698,29005
